In [10]:
import os
import webbrowser
import urllib.parse
import re
from openai import OpenAI
from config import API_KEY, ASSISTANT_ID
client = OpenAI(api_key=API_KEY)

### Test 1: We have a list of locations

In [6]:
def create_empty_thread():
    empty_thread = client.beta.threads.create()
    return empty_thread.id

def send_and_run():
    """
    Sends a user's message to the Assistant and retrieves the response within a thread,
    expecting responses about landmarks.
    """

    id = create_empty_thread()
    print("Chat with the Assistant! Type 'quit' to exit.")

    while True:
        os.system('cls')  # Clears the screen to keep the chat readable

        user_input = input("User: ")
        if user_input.lower() == 'quit':
            break

        # Sending a message to the thread
        thread_message = client.beta.threads.messages.create(
            id,
            role="user",
            content=user_input
        )

        # Running the thread with the specific assistant
        run = client.beta.threads.runs.create(
            id,
            assistant_id=ASSISTANT_ID
        )

        # Process and display messages
        thread_data = client.beta.threads.messages.list(id).data
        for msg in thread_data[::-1]:
            text, landmark_name = extract_landmark_name(msg.content[0].text.value)
            if msg.role == 'user':
                print(f"User: {text}")
            else:
                print(f"Assistant: {text}")
            if landmark_name:
                open_landmark_in_google_maps(landmark_name)  # Open landmark in Google Maps
        print("\n")

def extract_landmark_name(text):
    """
    Extracts the name of a landmark from the text.
    This is a placeholder and should be adapted to the expected response structure.
    """
    # For example, if the landmark name is expected to be explicitly stated:
    landmark_names = ["Statue of Liberty", "Eiffel Tower", "Great Wall of China"]  # Add more as needed
    for name in landmark_names:
        if name in text:
            # Clean the text by removing the landmark name
            clean_text = text.replace(name, '')
            return clean_text, name
    return text, None

def open_landmark_in_google_maps(landmark_name):
    """
    Opens the given landmark in Google Maps using the default web browser.
    """
    encoded_landmark = urllib.parse.quote(landmark_name)
    url = f"https://www.google.com/maps/search/?api=1&query={encoded_landmark}"
    webbrowser.open(url)
    print(f"Opening {landmark_name} in Google Maps...")

In [7]:
send_and_run()

Chat with the Assistant! Type 'quit' to exit.
User: hello


User: hello
Assistant: Hello! How can I assist you today? Are you looking for information on a specific location or planning a trip?
User: my name is Bob


User: hello
Assistant: Hello! How can I assist you today? Are you looking for information on a specific location or planning a trip?
User: my name is Bob
Assistant: Nice to meet you, Bob! How can I help you today?
User: tell me about the statue of liberty


User: hello
Assistant: Hello! How can I assist you today? Are you looking for information on a specific location or planning a trip?
User: my name is Bob
Assistant: Nice to meet you, Bob! How can I help you today?
User: tell me about the statue of liberty
Assistant: The  is a famous and iconic symbol of freedom and democracy located in New York Harbor in the United States. Here are some key details about the :

- The  was a gift from the people of France to the United States and was designed by French sculptor Frédéric A

KeyboardInterrupt: Interrupted by user

### Test 2: We do not have pre-defined locations

In [17]:
def create_empty_thread():
    empty_thread = client.beta.threads.create()
    return empty_thread.id

def send_and_run():
    """
    Sends a user's message to the Assistant and retrieves the response within a thread,
    expecting coordinates in the response that refer to landmarks.
    """

    id = create_empty_thread()
    print("Chat with the Assistant! Type 'quit' to exit.")

    while True:

        user_input = input("User: ")
        if user_input.lower() == 'quit':
            break

        # Sending a message to the thread
        thread_message = client.beta.threads.messages.create(
            id,
            role="user",
            content='''If you mention a landmark, location, business or anything of the sort, include at the end of your response its latitude and longitude according to this regex pattern: "\(([-+]?[0-9]*\.?[0-9]+),\s*([-+]?[0-9]*\.?[0-9]+)\)"''' + user_input
        )

        # Running the thread with the specific assistant
        run = client.beta.threads.runs.create(
            id,
            assistant_id=ASSISTANT_ID
        )

        # Process and display messages
        thread_data = client.beta.threads.messages.list(id).data
        for msg in thread_data[::-1]:
            text, coordinates = extract_coordinates(msg.content[0].text.value)
            if msg.role == 'user':
                print(f"User: {text}") 
            else:
                print(f"Assistant: {text}") 
            if coordinates:
                open_in_google_maps(coordinates)  # Open location in Google Maps
        print('\n')

def extract_coordinates(text):
    """
    Extracts coordinates from the text, assuming they are formatted as (latitude, longitude).
    """
    # Regex to find coordinates in the format (latitude, longitude)
    pattern = re.compile(r"\(([-+]?[0-9]*\.?[0-9]+),\s*([-+]?[0-9]*\.?[0-9]+)\)")
    match = pattern.search(text)
    if match:
        # Clean the text by removing the coordinates
        clean_text = pattern.sub("", text)
        coordinates = match.groups()
        return clean_text, coordinates
    return text, None

def open_in_google_maps(coordinates):
    """
    Opens the given coordinates in Google Maps using the default web browser.
    """
    latitude, longitude = coordinates
    url = f"https://www.google.com/maps/?q={latitude},{longitude}"
    webbrowser.open(url)
    print(f"Opening location in Google Maps: {latitude}, {longitude}")

In [18]:
send_and_run()

Chat with the Assistant! Type 'quit' to exit.
User: If you mention a landmark, location, business or anything of the sort, include at the end of your response its latitude and longitude according to this regex pattern: "\(([-+]?[0-9]*\.?[0-9]+),\s*([-+]?[0-9]*\.?[0-9]+)\)"hello


User: If you mention a landmark, location, business or anything of the sort, include at the end of your response its latitude and longitude according to this regex pattern: "\(([-+]?[0-9]*\.?[0-9]+),\s*([-+]?[0-9]*\.?[0-9]+)\)"hello
Assistant: Hello! How can I assist you today?
User: If you mention a landmark, location, business or anything of the sort, include at the end of your response its latitude and longitude according to this regex pattern: "\(([-+]?[0-9]*\.?[0-9]+),\s*([-+]?[0-9]*\.?[0-9]+)\)"my name is Bob


User: If you mention a landmark, location, business or anything of the sort, include at the end of your response its latitude and longitude according to this regex pattern: "\(([-+]?[0-9]*\.?[0-9]